In [23]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [24]:
spark = SparkSession.builder.appName("example").getOrCreate()

24/09/17 21:10:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [25]:
# Crear la sesión de Spark
spark = SparkSession.builder.appName("ClasificacionUrbanoRural").getOrCreate()

24/09/17 21:10:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [26]:
# Cargar el dataset principal con POBLACIÓN DANE
df_poblacion = spark.read.csv('data/Internet_Fijo_Penetraci_n_Municipio.csv', header=True, inferSchema=True,encoding='UTF-8')


In [27]:
# 2. Cargar los datasets de los departamentos con las áreas de terreno
df_departamentos = spark.read.csv('data/deptos/*.csv', header=True, inferSchema=True, encoding='UTF-8')

In [28]:
# 3. Convertir la columna AREA_TERRENO a tipo double (ya que es string)
df_departamentos = df_departamentos.withColumn('AREA_TERRENO', F.col('AREA_TERRENO').cast('double'))

In [29]:
# 4. Convertir el área de metros cuadrados a kilómetros cuadrados
df_departamentos = df_departamentos.withColumn('AREA_KM2', F.col('AREA_TERRENO') / 1000000)

In [30]:
# 5. Sumar el área total por municipio
df_areas_municipio = df_departamentos.groupBy('DEPARTAMENTO', 'MUNICIPIO').agg(F.sum('AREA_KM2').alias('AREA_TOTAL_KM2'))

In [31]:
# 6. Verificar duplicados en los datasets

# Verificar duplicados en el dataset de población
print("Verificando duplicados en el dataset de población:")
df_poblacion.groupBy('COD_DEPARTAMENTO', 'COD_MUNICIPIO').count().filter('count > 1').show()

Verificando duplicados en el dataset de población:
+----------------+-------------+-----+
|COD_DEPARTAMENTO|COD_MUNICIPIO|count|
+----------------+-------------+-----+
|              54|        54820|   12|
|              15|        15837|   12|
|              86|        86885|   12|
|              19|        19824|   12|
|               5|         5642|   12|
|              13|        13670|   12|
|              15|        15507|   12|
|              68|        68190|   12|
|              86|        86568|   12|
|              27|        27073|   12|
|              13|        13433|   12|
|              27|        27245|   12|
|              15|        15720|   12|
|               5|         5674|   12|
|              23|        23555|   12|
|              15|        15798|   12|
|              25|        25873|   12|
|              19|        19355|   12|
|              25|        25740|   12|
|               8|         8520|   12|
+----------------+-------------+-----+
only showing 

In [32]:
# Verificar duplicados en el dataset de áreas
print("Verificando duplicados en el dataset de áreas:")
df_areas_municipio.groupBy('DEPARTAMENTO', 'MUNICIPIO').count().filter('count > 1').show()

Verificando duplicados en el dataset de áreas:


+------------+---------+-----+
|DEPARTAMENTO|MUNICIPIO|count|
+------------+---------+-----+
+------------+---------+-----+



In [33]:
# 7. Renombrar las columnas para que coincidan con el dataset de población
df_areas_municipio = df_areas_municipio.withColumnRenamed('DEPARTAMENTO', 'COD_DEPARTAMENTO').withColumnRenamed('MUNICIPIO', 'COD_MUNICIPIO')

In [34]:
# 8. Asegurarse de que los códigos de departamento y municipio sean del mismo tipo (string)
df_poblacion = df_poblacion.withColumn('COD_DEPARTAMENTO', F.col('COD_DEPARTAMENTO').cast('string'))
df_poblacion = df_poblacion.withColumn('COD_MUNICIPIO', F.col('COD_MUNICIPIO').cast('string'))

In [35]:
df_areas_municipio = df_areas_municipio.withColumn('COD_DEPARTAMENTO', F.col('COD_DEPARTAMENTO').cast('string'))
df_areas_municipio = df_areas_municipio.withColumn('COD_MUNICIPIO', F.col('COD_MUNICIPIO').cast('string'))

In [36]:
# 9. Verificar qué municipios de población no tienen área correspondiente
municipios_faltantes = df_poblacion.join(df_areas_municipio, on=['COD_DEPARTAMENTO', 'COD_MUNICIPIO'], how='left_anti')

In [43]:
print("Municipios faltantes en el dataset de áreas:")
municipios_faltantes.select('COD_DEPARTAMENTO', 'COD_MUNICIPIO', 'MUNICIPIO').show(1000, truncate=False)

Municipios faltantes en el dataset de áreas:


+----------------+-------------+--------------------+
|COD_DEPARTAMENTO|COD_MUNICIPIO|MUNICIPIO           |
+----------------+-------------+--------------------+
|54              |54250        |EL TARRA            |
|25              |25594        |QUETAME             |
|5               |5142         |CARACOLI            |
|25              |25594        |QUETAME             |
|25              |25594        |QUETAME             |
|54              |54250        |EL TARRA            |
|54              |54250        |EL TARRA            |
|5               |5142         |CARACOLI            |
|5               |5142         |CARACOLI            |
|25              |25594        |QUETAME             |
|54              |54250        |EL TARRA            |
|5               |5142         |CARACOLI            |
|54              |54250        |EL TARRA            |
|5               |5142         |CARACOLI            |
|5               |5142         |CARACOLI            |
|54              |54250     

In [38]:
# 10. Unir el dataset de áreas con el dataset de población usando los códigos
df_completo = df_areas_municipio.join(df_poblacion, on=['COD_DEPARTAMENTO', 'COD_MUNICIPIO'], how='inner')

In [39]:
# 11. Calcular la densidad poblacional
df_completo = df_completo.withColumn('densidad_poblacional', F.col('POBLACIÓN DANE') / F.col('AREA_TOTAL_KM2'))

In [40]:
# 12. Clasificar como Urbano o Rural según la densidad poblacional
df_completo = df_completo.withColumn(
    'Clasificacion',
    F.when(F.col('densidad_poblacional') >= 100, 'Urbano').otherwise('Rural')
)

In [41]:
# 13. Verificar el número total de municipios en el dataset completo
num_municipios = df_completo.select('COD_MUNICIPIO').distinct().count()
print(f"Número total de municipios en el dataset completo: {num_municipios}")

Número total de municipios en el dataset completo: 832


In [42]:
# 14. Guardar el nuevo dataset clasificado en la carpeta transformaciones
df_completo.write.csv('data/deptos/transformaciones/transformaciones.csv', header=True)

In [44]:
municipios_faltantes_data = municipios_faltantes.select('COD_DEPARTAMENTO', 'COD_MUNICIPIO', 'MUNICIPIO') \
    .write.csv('data/deptos/transformaciones/municipios_faltantes.csv', header=True)